In [1]:
import os
import re
import datetime
from pathlib import Path

import pandas as pd
import numpy as np
import shapefile
import pupygrib
import ogr
from osgeo import gdal
from netCDF4 import Dataset

In [2]:
class Analysis:
    ELEMENTS = ['O3', 'NO2', 'CO']
    HEIGHT_DEVIATION = 500
    HEIGHT_MAX = 5000

    def __init__(self):
        self.project_folder = Path(os.path.abspath('')).parent
        self.data_folder = self.project_folder / 'data'
        self.shp_folder = self.data_folder / 'shp/crop'
        self.nc_folder = self.data_folder / 'nc'
        
    def getDate(self, file_path):
        m = re.search(r"\d{8}", file_path)
        date = datetime.datetime.strptime(m.group(), '%Y%m%d')
        
        return date
        
    def process(self, file_, date):
        date_str = date.strftime("%Y%m%d")
        with shapefile.Reader(str(file_)) as shp_file:
            result_row = {}
            result_row['date'] = date_str

            records = shp_file.records()
            fields = shp_file.fields[1:]
            fields_names = [field[0] for field in fields]
            
            fields_names[3] = 'O3'  # rename Ozone to O3

            data = pd.DataFrame(
                np.array(records), columns=fields_names
            )
            data[['StdAltitu']] /= 3.281  # ft to meters

            points_x = np.empty(len(records))
            points_y = np.empty(len(records))

            shp = ogr.Open(str(file_.absolute()))
            layer = shp.GetLayer()
            i = 0
            while i < len(records):
                point = layer.GetFeature(i)
                geom = point.GetGeometryRef()
                points_x[i] = geom.GetPoint(0)[0]
                points_y[i] = geom.GetPoint(0)[1]
                i += 1

            data['lat'] = points_x
            data['long'] = points_y

            for element in self.ELEMENTS:
                data_clean = data.dropna(axis=1, how='all')
                if element not in data_clean.columns:
                    continue

                data_clean = data_clean.dropna(how='any', axis=0, subset=[element])

                clean_data_2km = pd.DataFrame()
                clean_data_3km = pd.DataFrame()
                clean_data_5km = pd.DataFrame()

                for index, row in data_clean.iterrows():
                    if row['StdAltitu'] > self.HEIGHT_MAX + self.HEIGHT_DEVIATION:
                        continue

                    nc_date = datetime.datetime.combine(date, datetime.datetime.min.time())
                    nc_date += datetime.timedelta(seconds=row['TimeCRef'])
                    nc_date_str = nc_date.strftime("%Y%m%d")

                    nc = 0
                    for file_ in self.nc_folder.iterdir():
                        if nc_date_str in file_.name and element in file_.name:
                            with Dataset(str(file_), 'r', format='NETCDF4') as nc_:
                                nc = nc_
                                el = nc.variables[element]
                                lons = nc.variables['lon'][:]
                                lats = nc.variables['lat'][:]
                                heights = nc.variables['height'][:]

                                # latitude index
                                lat_id = np.argmin( np.abs( lats - row['lat'] ) )
                                # longitude index
                                lon_id = np.argmin( np.abs( lons - row['long'] ) )
                                # height index
                                if np.abs( heights - row['StdAltitu'] ).any() > self.HEIGHT_DEVIATION:
                                    continue
                                else:
                                    height_id = np.argmin( np.abs( heights - row['StdAltitu'] ) )

                                # time index
                                nc_hour = nc_date.strftime("%H")
                                
                                row_value = {
                                    'nc_data_'+element: float(el[np.int64(nc_hour), height_id, lat_id, lon_id]),
                                    element: row[element]
                                }

                                if height_id == 5:  # if 2km
                                    clean_data_2km = clean_data_2km.append(row_value, ignore_index=True)
                                elif height_id == 6: # if 3km
                                    clean_data_3km = clean_data_3km.append(row_value, ignore_index=True)
                                elif height_id == 7: # if 5km
                                    clean_data_5km = clean_data_5km.append(row_value, ignore_index=True)
                                else:
                                    continue
                    if not nc:
                        continue

                if not clean_data_2km.empty:
                    corr_2km = clean_data_2km['nc_data_' + element].corr(clean_data_2km[element])
                    result_row[element+'_2km'] = corr_2km
                    result_row['dots_shp_'+element+'_2km'] = len(clean_data_2km.index)
                    result_row['dots_nc_'+element+'_2km'] = clean_data_2km['nc_data_' + element].nunique()
                if not clean_data_3km.empty:
                    corr_3km = clean_data_3km['nc_data_' + element].corr(clean_data_3km[element])
                    result_row[element+'_3km'] = corr_3km
                    result_row['dots_shp_'+element+'_3km'] = len(clean_data_3km.index)
                    result_row['dots_nc_'+element+'_3km'] = clean_data_3km['nc_data_' + element].nunique()
                if not clean_data_5km.empty:
                    corr_5km = clean_data_5km['nc_data_' + element].corr(clean_data_5km[element])
                    result_row[element+'_5km'] = corr_5km
                    result_row['dots_shp_'+element+'_5km'] = len(clean_data_5km.index)
                    result_row['dots_nc_'+element+'_5km'] = clean_data_5km['nc_data_' + element].nunique()
        return result_row

    def analyse(self, date: datetime.date = None):
        # Get data
        result_data = pd.DataFrame([], columns=['date'])
        data = []
        if date:
            date_str = date.strftime("%Y%m%d")
            for file_ in self.shp_folder.iterdir():
                if date_str in file_.name and file_.suffix == '.shp':
                    result_row = self.process(file_, date)      
                    result_data = result_data.append(result_row, ignore_index=True)
        else:
            for file_ in self.shp_folder.iterdir():
                if file_.suffix == '.shp':
                    date = self.getDate(str(file_))
                    result_row = self.process(file_, date)      
                    result_data = result_data.append(result_row, ignore_index=True)
                
        return result_data

In [3]:
a = Analysis()
d = a.analyse()
print(d)

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


        date  CO_3km  CO_5km  O3_2km  O3_3km  O3_5km  dots_nc_CO_3km  \
0   20151201     NaN     NaN     NaN     NaN     NaN             1.0   
1   20151130     NaN     NaN     NaN     NaN     NaN             NaN   
2   20160114     NaN     NaN     NaN     NaN     NaN             NaN   
3   20160215     NaN     NaN     NaN     NaN     NaN             NaN   
4   20160113     NaN     NaN     NaN     NaN     NaN             NaN   
5   20160112     NaN     NaN     NaN     NaN     NaN             NaN   
6   20160215     NaN     NaN     NaN     NaN     NaN             1.0   
7   20160216     NaN     NaN     NaN     NaN     NaN             1.0   
8   20160113     NaN     NaN     NaN     NaN     NaN             NaN   
9   20151201     NaN     NaN     NaN     NaN     NaN             NaN   
10  20151202     NaN     NaN     NaN     NaN     NaN             NaN   

    dots_nc_CO_5km  dots_nc_O3_2km  dots_nc_O3_3km  dots_nc_O3_5km  \
0              1.0             1.0             1.0             1.